Base Path for the Code Repository

In [1]:
base_path = "/content/drive/MyDrive"

In [2]:
import warnings

# Suppress all warnings
warnings.filterwarnings('ignore')

# Suppress only FutureWarnings
warnings.filterwarnings('ignore', category=FutureWarning)

# Suppress only DeprecationWarnings
warnings.filterwarnings('ignore', category=DeprecationWarning)


In [3]:
import pandas as pd
import numpy as np
import os
import shutil

pd.set_option('display.max_columns', None)

# Loading the demographic tables



In [6]:
df_admissions = pd.read_csv(f"{base_path}/BM7/Dissertation/Datasets/admissions.csv")
df_patients = pd.read_csv(f"{base_path}/BM7/Dissertation/Datasets/patients.csv")
df_icustays = pd.read_csv(f"{base_path}/BM7/Dissertation/Datasets/icustays.csv")

## 1. Patients Table

In [ ]:
df_patients.head()

In [ ]:
# Data shape
print("Patients Data Shape:")
print(df_patients.shape)  # Number of rows and columns

In [ ]:
# Basic statistics
patients_stats = df_patients.describe()

# Check for null values
patients_null = df_patients.isnull().sum()

# Unique values
patients_unique = df_patients.nunique()

print("\nPatients Statistics:")
print(patients_stats)
print("\nPatients Null Values:")
print(patients_null)
print("\nPatients Unique Values:")
print(patients_unique)

### Patients : Data Pre-processing

In [ ]:
# Convert DOB and DOD to datetime
df_patients['DOB'] = pd.to_datetime(df_patients['DOB'], errors='coerce')
df_patients['DOD'] = pd.to_datetime(df_patients['DOD'], errors='coerce')

In [ ]:
# Convert gender to numerical
df_patients['GENDER'] = df_patients['GENDER'].map({'M': 1, 'F': 0})

In [ ]:
# Calculate age at death
df_patients['AGE_AT_DEATH'] = np.where(df_patients['DOD'].isnull(),
                                       np.nan,
                                       (df_patients['DOD'].dt.year - df_patients['DOB'].dt.year))

In [ ]:
df_patients.head(10)

In [ ]:
df_patients.shape

# 2. Admissions Table

In [ ]:
df_admissions.head()

In [ ]:
# Basic statistics
admissions_stats = df_admissions.describe()

# Check for null values
admissions_null = df_admissions.isnull().sum()

# Unique values
admissions_unique = df_admissions.nunique()

# Data shape
print("Admissions Data Shape:")

print(df_admissions.shape)  # Number of rows and columns
print("\nAdmissions Statistics:")
print(admissions_stats)
print("\nAdmissions Null Values:")
print(admissions_null)
print("\nAdmissions Unique Values:")
print(admissions_unique)

## Admissions : Data Pre-Processing

In [ ]:
# Data preprocessing
# Convert ADMITTIME, DISCHTIME, DEATHTIME to datetime
df_admissions['ADMITTIME'] = pd.to_datetime(df_admissions['ADMITTIME'], errors='coerce')
df_admissions['DISCHTIME'] = pd.to_datetime(df_admissions['DISCHTIME'], errors='coerce')
df_admissions['DEATHTIME'] = pd.to_datetime(df_admissions['DEATHTIME'], errors='coerce')

In [ ]:
# Fill missing values in categorical columns with 'Unknown'
categorical_columns = ['ADMISSION_TYPE', 'ADMISSION_LOCATION', 'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY']
df_admissions[categorical_columns] = df_admissions[categorical_columns].fillna('Unknown')

In [ ]:
df_admissions.head()

# 3. ICUStays Table

In [ ]:
df_icustays.head(10)

In [ ]:
# Basic statistics
icustays_stats = df_icustays.describe()

# Check for null values
icustays_null = df_icustays.isnull().sum()

# Unique values
icustays_unique = df_icustays.nunique()

# Data shape
print("ICU Stays Data Shape:")
print(df_icustays.shape)  # Number of rows and columns
print("\nICU Stays Statistics:")
print(icustays_stats)
print("\nICU Stays Null Values:")
print(icustays_null)
print("\nICU Stays Unique Values:")
print(icustays_unique)


## ICUStays : Data Pre-Processing

In [ ]:
 # Data preprocessing
# Convert INTIME and OUTTIME to datetime
df_icustays['INTIME'] = pd.to_datetime(df_icustays['INTIME'], errors='coerce')
df_icustays['OUTTIME'] = pd.to_datetime(df_icustays['OUTTIME'], errors='coerce')

In [ ]:
df_icustays.head()

# Merging Demographic dataset

In [ ]:
# Step 1: Merge PATIENTS and ADMISSIONS on SUBJECT_ID
merged_data = pd.merge(df_patients, df_admissions, on='SUBJECT_ID', how='inner')

print("\nSample of Merged Data:")
merged_data.shape

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Assuming 'merged_data' is your dataframe containing the integrated data
# Load the merged data (update the path as per your environment)
# merged_data = pd.read_csv('path/to/your/merged_data.csv')

# Checking the top 10 most common ethnicities
top_ethnicities = merged_data['ETHNICITY'].value_counts().nlargest(10)

# Display the top 10 ethnicities
print("Top 10 Ethnicities in the Dataset:")
print(top_ethnicities)

# Plotting the top 10 ethnicities
plt.figure(figsize=(10, 6))
top_ethnicities.plot(kind='bar', color='skyblue')
plt.title('Top 10 Ethnicities in the Dataset')
plt.xlabel('Ethnicity')
plt.ylabel('Number of Patients')
plt.xticks(rotation=90)
plt.show()


### Check/ Drop Duplicates

In [ ]:
dup = merged_data.duplicated(subset=['HADM_ID']).any()
dup

Drop duplicate 'HADM_ID' entries, keeping the first occurrence

In [ ]:
# Drop duplicate 'HADM_ID' entries, keeping the first occurrence
merged_data_unique = merged_data.drop_duplicates(subset='HADM_ID')

In [ ]:
merged_data.shape

In [ ]:
merged_data.nunique()

Merge ICU table on HADM

In [ ]:
# Step 2: Merge with ICUSTAYS on HADM_ID
merged_data = pd.merge(merged_data, df_icustays, on=['HADM_ID'], how='inner')

In [ ]:
# Display the shape and sample of merged data
print("Final Merged Data Shape:")
print(merged_data.shape)

print("\nSample of Merged Data:")
merged_data.head()

### Drop duplicate HADMS

1. Merge the DataFrames: Merge df_patients and df_admissions on SUBJECT_ID.
2. Sort by HADM_ID and intime:
3. Drop Duplicates

In [ ]:
# Convert datetime columns to datetime format for proper sorting
merged_data['INTIME'] = pd.to_datetime(merged_data['INTIME'])

# Sort by HADM_ID and INTIME, and then drop duplicates based on HADM_ID, keeping the first entry
merged_data = merged_data.sort_values(by=['HADM_ID', 'INTIME']).drop_duplicates(subset='HADM_ID', keep='first')

merged_data.head()

In [ ]:
merged_data.shape

In [ ]:
merged_data.nunique()

## Hadm is lesse this time beacuse these are the patiesnts that went to ICU - 57786

## Drop Irrelevant/Redundant columns

In [ ]:
# Drop ROW_ID_y and SUBJECT_ID_y from merged_data
merged_data.drop(columns=['ROW_ID_y', 'ROW_ID','ROW_ID_x'], inplace=True)

# Rename ROW_ID_x and SUBJECT_ID_x to remove _x suffix
merged_data.rename(columns={ 'SUBJECT_ID_x': 'SUBJECT_ID'}, inplace=True)

In [ ]:
demographic_data = merged_data

# List of columns to drop
columns_to_drop = [
    'DOD_HOSP', 'DOD_SSN',  # Redundant or less relevant death information
    'EXPIRE_FLAG',  # If redundant with HOSPITAL_EXPIRE_FLAG or not needed
    'DBSOURCE', 'FIRST_CAREUNIT', 'LAST_CAREUNIT',  # ICU stay details
    'FIRST_WARDID', 'LAST_WARDID', 'INTIME', 'OUTTIME', #'LOS',  # More ICU stay details
    'HAS_CHARTEVENTS_DATA','EDREGTIME','EDOUTTIME','DISCHARGE_LOCATION','ADMISSION_LOCATION','DEATHTIME', # Admission details not important
    'LANGUAGE', 'RELIGION', 'MARITAL_STATUS', 'ETHNICITY', 'INSURANCE' # Irrelevant Demographic info
]

# Drop columns
demographic_data = demographic_data.drop(columns=columns_to_drop)

#Rename columns
demographic_data = demographic_data.rename(columns={'LOS': 'LOS_ICU'})

demographic_data.head()

#### Feature Addition: Age at admission

In [ ]:
# Calculate age at admission
def calculate_age_at_admission(dob, admittime):
    return admittime.year - dob.year - ((admittime.month, admittime.day) < (dob.month, dob.day))

demographic_data['AGE_AT_ADMISSION'] = demographic_data.apply(lambda row: calculate_age_at_admission(row['DOB'], row['ADMITTIME']), axis=1)

demographic_data.head()

In [ ]:
# Desired column order
desired_order = [
    'SUBJECT_ID', 'HADM_ID','GENDER', 'DOB', 'DOD', 'AGE_AT_ADMISSION' ,'AGE_AT_DEATH',
    'ADMITTIME', 'DISCHTIME', 'ADMISSION_TYPE',
    'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG', 'ICUSTAY_ID', 'LOS_ICU'
]

# Reorder the columns
demographic_data = demographic_data[desired_order]

In [ ]:
demographic_data.head(10)

In [ ]:
demographic_data.shape

In [ ]:
demographic_data.nunique()

### EDA Demographics Data

In [ ]:
df_demographic = demographic_data.copy()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Display summary statistics
print(df_demographic.describe())

# Check for missing values
print(df_demographic.isnull().sum())


In [ ]:
# Gender distribution
gender_counts = df_demographic['GENDER'].value_counts()

plt.figure(figsize=(6, 4))
sns.barplot(x=gender_counts.index, y=gender_counts.values, palette='pastel')
plt.title('Gender Distribution', fontsize=15)
plt.xlabel('Gender', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks([0, 1], ['Female', 'Male'])
plt.show()


4. Length of Stay in ICU (LOS_ICU)
Objective: To analyze the length of ICU stays to understand the severity and resource utilization of different patient groups.

In [ ]:
# Plot the distribution of ICU length of stay
plt.figure(figsize=(10, 5))
sns.histplot(df_demographic['LOS_ICU'], bins=30, kde=True, color='lightgreen')
plt.title('Distribution of ICU Length of Stay', fontsize=15)
plt.xlabel('Length of Stay in ICU (days)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.axvline(df_demographic['LOS_ICU'].mean(), color='red', linestyle='--', label='Mean LOS')
plt.legend()
plt.show()


In [ ]:
# Hospital mortality analysis
mortality_counts = df_demographic['HOSPITAL_EXPIRE_FLAG'].value_counts()

plt.figure(figsize=(6, 4))
sns.barplot(x=mortality_counts.index, y=mortality_counts.values, palette='muted')
plt.title('Hospital Mortality Distribution', fontsize=15)
plt.xlabel('Expired in Hospital (0=No, 1=Yes)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks([0, 1], ['No', 'Yes'])
plt.show()


**3. Mortality Analysis by Age and Gender**


In [ ]:
# Mortality by Age and Gender
plt.figure(figsize=(10, 5))
sns.boxplot(x='GENDER', y='AGE_AT_DEATH', data=demographic_data)
plt.title('Age at Death by Gender', fontsize=15)
plt.xlabel('Gender', fontsize=12)
plt.ylabel('Age at Death', fontsize=12)
plt.show()

# Mortality by Age Group
demographic_data['Age_Group'] = pd.cut(demographic_data['AGE_AT_ADMISSION'], bins=[0, 18, 35, 50, 65, 80, 100],
                                       labels=['0-18', '19-35', '36-50', '51-65', '66-80', '80+'])

mortality_by_age_group = demographic_data.groupby('Age_Group')['HOSPITAL_EXPIRE_FLAG'].mean()

plt.figure(figsize=(10, 5))
sns.barplot(x=mortality_by_age_group.index, y=mortality_by_age_group.values, palette='muted')
plt.title('Mortality Rate by Age Group', fontsize=15)
plt.xlabel('Age Group', fontsize=12)
plt.ylabel('Mortality Rate', fontsize=12)
plt.show()


In [ ]:
# Admission types distribution
admission_types = df_demographic['ADMISSION_TYPE'].value_counts()

plt.figure(figsize=(8, 5))
sns.barplot(x=admission_types.index, y=admission_types.values, palette='Set2')
plt.title('Admission Types Distribution', fontsize=15)
plt.xlabel('Admission Type', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Top diagnoses
top_diagnoses = df_demographic['DIAGNOSIS'].value_counts().head(10)

plt.figure(figsize=(10, 5))
sns.barplot(x=top_diagnoses.index, y=top_diagnoses.values, palette='coolwarm')
plt.title('Top 10 Diagnoses', fontsize=15)
plt.xlabel('Diagnosis', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Plot the distribution of age at admission
plt.figure(figsize=(10, 5))
sns.histplot(df_demographic['AGE_AT_ADMISSION'], bins=30, kde=True, color='skyblue')
plt.title('Distribution of Age at Admission', fontsize=15)
plt.xlabel('Age at Admission', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.axvline(df_demographic['AGE_AT_ADMISSION'].mean(), color='red', linestyle='--', label='Mean Age')
plt.legend()
plt.show()


Red dashed line represents the mean age, providing a visual cue about the average age in the dataset.

In [ ]:
# Generate word cloud for diagnosis descriptions
text = ' '.join(df_demographic['DIAGNOSIS'].dropna())

wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.title('Word Cloud of Diagnoses', fontsize=15)
plt.show()


### Export demographic dataframe to csv

In [ ]:
# demographic_data.to_csv(r'G:\My Drive\BM7\Dissertation\Datasets\demographic_modality.csv', index=False)
demographic_data.to_csv(f'{base_path}/BM7/Dissertation/Datasets/demographic_modality.csv', index=False)

In [7]:
# demographic_data = pd.read_csv(r'G:\My Drive\BM7\Dissertation\Datasets\demographic_modality.csv')
demographic_data = pd.read_csv(f'{base_path}/BM7/Dissertation/Datasets/demographic_modality.csv')

In [ ]:
demographic_data.shape

In [ ]:
# List of categorical columns to check
categorical_columns = [
    'GENDER', 'ADMISSION_TYPE',
]

# Function to get unique values for each categorical column
def print_unique_values(df, columns):
    for column in columns:
        unique_values = df[column].unique()
        print(f"Unique values in column '{column}':")
        print(unique_values)
        print("\n")

# Print unique values for categorical columns in demographic_data
print_unique_values(demographic_data, categorical_columns)
